In [44]:
%load_ext autoreload
%autoreload 2

# %set_env CUDA_VISIBLE_DEVICES=7
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.predict import Predict
import dsp
from dsp.primitives.vision import Image
from dsp.modules.gpt4vision import GPT4Vision
from dsp.modules.cache_utils import CacheMemory

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1) Configure the VLM

In [45]:
gpt = GPT4Vision()
dspy.settings.configure(lm=gpt)

### 2) Ask questions about the scene

In [49]:
#Define a simple signature for basic question answering
class BasicVQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    image = dspy.InputField()
    answer = dspy.OutputField(desc="x=?, y=?, z=?, roll=?, pitch=?, yaw=?, open_or_close_gripper=?")

gpt = GPT4Vision()
dspy.settings.configure(lm=gpt, num_generations=1)
#Pass signature to ChainOfThought module
generate_answer = dspy.Predict(BasicVQA)

# Call the predictor on a particular input alongside a hint.
question='What action should the robot take to pick up the block?'
image= Image('gripper_mid_to_grasp.png')
hint = "Usually, the depth is underestimated."
pred = generate_answer(question=question, image=image, hint=hint)


print(f"Question: {question}")
print(f"Predicted Answer: {pred.answer}")
gpt.inspect_history(n=10)
image.pil.show()

gripper_mid_to_grasp.png
Question: What action should the robot take to pick up the block?
Predicted Answer: To pick up the block, the robot should perform the following actions:

1. Approach the block: The robot should move its arm towards the block, positioning the gripper directly above it.

2. Align the gripper: The robot should adjust the orientation of its gripper so that it is aligned with the block for an optimal grasp.

3. Open the gripper: If the gripper is not already open, the robot should open it to prepare for grasping the block.

4. Lower the gripper: The robot should carefully lower the gripper until it is close enough to the block without touching the surface or the block itself.

5. Close the gripper: Once the gripper is in the correct position, the robot should close




Question: What action should the robot take to pick up the block? ... To pick up the block, the robot should perform the following actions:

1. Approach the block: The robot should move its arm towar